In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import math
import json
import pickle
import tensorflow.python.keras as keras
from keras.layers import Input, Layer, LeakyReLU, BatchNormalization, Conv2D, MaxPooling2D, UpSampling2D, Concatenate, Add, Lambda
from keras.models import Model, load_model, model_from_json, clone_model
from keras.optimizers import SGD, Adam
import keras.backend as K
from PIL import Image
import os
K.clear_session()
'''
# file h5 dibuat dengan convert.py dari https://github.com/awe777/keras-yolo3
model0 = load_model("yolov3_tiny.h5")   # dibuat dari kode sebelum fork
model1 = load_model("yolov3_tiny_.h5")  # dibuat dari kode setelah diubah di fork pribadi
print(model0.to_json() == model1.to_json())
print(model.summary())
# '''
print()

Using TensorFlow backend.


In [2]:
def roundingAlgo(x):
    return K.round(x)

class RoundQinf_4(Layer):
    def __init__(self, **kwargs):
        super(RoundQinf_4, self).__init__(**kwargs)
        self.trainable = False
    def build(self, input_shape):
        super(RoundQinf_4, self).build(input_shape)
    def call(self, X):
        return roundingAlgo(X * 16) / 16.0
    def get_config(self):
        base_config = super(RoundQinf_4, self).get_config()
        return dict(list(base_config.items()))
class RoundOverflowQ7_12(Layer):
    def __init__(self, **kwargs):
        super(RoundOverflowQ7_12, self).__init__(**kwargs)
        self.trainable = False
    def build(self, input_shape):
        super(RoundOverflowQ7_12, self).build(input_shape)
    def call(self, X):
        return (((roundingAlgo(X * 4096) + 524288) % 1048576) - 524288) / 4096.0
    def get_config(self):
        base_config = super(RoundOverflowQ7_12, self).get_config()
        return dict(list(base_config.items()))
class RoundQx_8minusx(Layer):
    def __init__(self, **kwargs):
        super(RoundQx_8minusx, self).__init__(**kwargs)
        self.trainable = False
    def build(self, input_shape):
        super(RoundQx_8minusx, self).build(input_shape)
    def call(self, X):
        pow_2_to_expplus8 = 2 ** (8 + (K.log(K.abs(X)) / math.log(2)) // 1)
        return roundingAlgo(X * pow_2_to_expplus8) / (pow_2_to_expplus8 * 1.0)
    def get_config(self):
        base_config = super(RoundQx_8minusx, self).get_config()
        return dict(list(base_config.items()))
class RoundOverflowQ3_4(Layer):
    def __init__(self, **kwargs):
        super(RoundOverflowQ3_4, self).__init__(**kwargs)
        self.trainable = False
    def build(self, input_shape):
        super(RoundOverflowQ3_4, self).build(input_shape)
    def call(self, X):
        return (((roundingAlgo(X * 16) + 128) % 256) - 128) / 16.0
    def get_config(self):
        base_config = super(RoundOverflowQ3_4, self).get_config()
        return dict(list(base_config.items()))
class RoundOverflowQ7_0(Layer):
    def __init__(self, **kwargs):
        super(RoundOverflowQ7_0, self).__init__(**kwargs)
        self.trainable = False
    def build(self, input_shape):
        super(RoundOverflowQ7_0, self).build(input_shape)
    def call(self, X):
        return ((roundingAlgo(X) + 128) % 256) - 128
    def get_config(self):
        base_config = super(RoundOverflowQ7_0, self).get_config()
        return dict(list(base_config.items()))
class Identity(Layer):
    def __init__(self, **kwargs):
        super(Identity, self).__init__(**kwargs)
        self.trainable = False
    def call(self, X):
        return X
    def get_config(self):
        base_config = super(Identity, self).get_config()
        return dict(list(base_config.items()))
class IdentityFinalLayer(Layer):
    def __init__(self, **kwargs):
        super(IdentityFinalLayer, self).__init__(**kwargs)
        self.trainable = False
    def call(self, X):
        return X
    def get_config(self):
        base_config = super(IdentityFinalLayer, self).get_config()
        return dict(list(base_config.items()))

print("Custom layer classes successfully defined")

Custom layer classes successfully defined


In [3]:
customObjects = {
    'Identity': Identity,
    'RoundQinf_4': RoundQinf_4,
    'RoundOverflowQ3_4': RoundOverflowQ3_4,
    'RoundQx_8minusx': RoundQx_8minusx,
    'RoundOverflowQ7_12': RoundOverflowQ7_12,
    'RoundOverflowQ7_0': RoundOverflowQ7_0,
    'IdentityFinalLayer': IdentityFinalLayer
}

In [4]:
def prepare_image(image_dir, input_shape):
    # from PIL import Image
    # import numpy as np
    image_data = 0
    h, w = input_shape
    try:
        image = Image.open(image_dir)
        iw, ih = image.size
        scale = min(w/iw, h/ih)
        nw = int(iw*scale)
        nh = int(ih*scale)
        dx = (w-nw)//2
        dy = (h-nh)//2
        image = image.resize((nw,nh), Image.BICUBIC)
        new_image = Image.new('RGB', (w,h), (128,128,128))
        new_image.paste(image, (dx, dy))
        image_data = np.array(new_image)/255.
        return image_data
    except:
        return None
# TO DO: implement yolo_head in order to get human-understandable result from model output

def get_anchors(anchors_path):
    '''loads the anchors from a file'''
    with open(anchors_path) as f:
        anchors = f.readline()
    anchors = [float(x) for x in anchors.split(',')]
    return np.array(anchors).reshape(-1, 2)

tinyYolo_anchors = get_anchors("../CNN-VLSI/tiny_yolo_anchors.txt")

def yolo_head_nonLoss(feats, anchors, num_classes, input_shape):
    """Convert final layer features to bounding box parameters."""
    num_anchors = len(anchors)
    # Reshape to batch, height, width, num_anchors, box_params.
    anchors_tensor = K.reshape(K.constant(anchors), [1, 1, 1, num_anchors, 2])

    grid_shape = K.shape(feats)[1:3] # height, width
    grid_y = K.tile(K.reshape(K.arange(0, stop=grid_shape[0]), [-1, 1, 1, 1]), [1, grid_shape[1], 1, 1])
    grid_x = K.tile(K.reshape(K.arange(0, stop=grid_shape[1]), [1, -1, 1, 1]), [grid_shape[0], 1, 1, 1])
    grid = K.concatenate([grid_x, grid_y])
    grid = K.cast(grid, 'float32') # K.cast(grid, K.dtype(feats))

    feats = K.reshape(
        feats, [-1, grid_shape[0], grid_shape[1], num_anchors, num_classes + 5])

    # Adjust preditions to each spatial grid point and anchor size.
    # edited by instructions in https://stackoverflow.com/questions/57558476/training-a-keras-model-yields-multiple-optimizer-errors
    box_xy = (K.sigmoid(feats[..., :2]) + grid) / K.cast(grid_shape[...,::-1], K.dtype(feats))
    box_wh = K.exp(feats[..., 2:4]) * anchors_tensor / K.cast(input_shape[...,::-1], K.dtype(feats))
    box_confidence = K.sigmoid(feats[..., 4:5])
    box_class_probs = K.sigmoid(feats[..., 5:])

    return box_xy, box_wh, box_confidence, box_class_probs

def yolo_correct_boxes(box_xy, box_wh, input_shape, image_shape):
    '''Get corrected boxes'''
    box_yx = box_xy[..., ::-1]
    box_hw = box_wh[..., ::-1]
    input_shape = K.cast(input_shape, K.dtype(box_yx))
    image_shape = K.cast(image_shape, K.dtype(box_yx))
    new_shape = K.round(image_shape * K.min(input_shape/image_shape))
    offset = (input_shape-new_shape)/2./input_shape
    scale = input_shape/new_shape
    box_yx = (box_yx - offset) * scale
    box_hw *= scale

    box_mins = box_yx - (box_hw / 2.)
    box_maxes = box_yx + (box_hw / 2.)
    boxes =  K.concatenate([
        box_mins[..., 0:1],  # y_min
        box_mins[..., 1:2],  # x_min
        box_maxes[..., 0:1],  # y_max
        box_maxes[..., 1:2]  # x_max
    ])

    # Scale boxes back to original image shape.
    boxes *= K.concatenate([image_shape, image_shape])
    return boxes

def yolo_boxes_and_scores(feats, anchors, num_classes, input_shape, image_shape):
    '''Process Conv layer output'''
    box_xy, box_wh, box_confidence, box_class_probs = yolo_head_nonLoss(feats,
        anchors, num_classes, input_shape)
    boxes = yolo_correct_boxes(box_xy, box_wh, input_shape, image_shape)
    boxes = K.reshape(boxes, [-1, 4])
    box_scores = box_confidence * box_class_probs
    box_scores = K.reshape(box_scores, [-1, num_classes])
    return boxes, box_scores

def yolo_eval(yolo_outputs,
              anchors,
              num_classes,
              image_shape,
              max_boxes=20,
              score_threshold=.6,
              iou_threshold=.5):
    """Evaluate YOLO model on given input and return filtered boxes."""
    num_layers = len(yolo_outputs)
    anchor_mask = [[6,7,8], [3,4,5], [0,1,2]] if num_layers==3 else [[3,4,5], [1,2,3]] # default setting
    input_shape = K.shape(yolo_outputs[0])[1:3] * 32
    boxes = []
    box_scores = []
    for l in range(num_layers):
        _boxes, _box_scores = yolo_boxes_and_scores(yolo_outputs[l],
            anchors[anchor_mask[l]], num_classes, input_shape, image_shape)
        boxes.append(_boxes)
        box_scores.append(_box_scores)
    boxes = K.concatenate(boxes, axis=0)
    box_scores = K.concatenate(box_scores, axis=0)
#     print("Box tensor:", boxes.shape)
#     print(K.get_value(boxes))
#     print("Box score tensor:", box_scores.shape)
#     print(K.get_value(box_scores))
#     print("class index of maximum prob value:", K.argmax(box_scores, axis=1).shape)
#     print(K.get_value(K.argmax(box_scores, axis=1)))
#     print("maximum prob value: ",K.max(box_scores, axis=1).shape)
#     print(K.get_value(K.max(box_scores, axis=1)))
#     print("maximum prob mask: ", (K.max(box_scores, axis=1) >= score_threshold).shape)
#     print(K.get_value(K.max(box_scores, axis=1) >= score_threshold)) # 0.027))
#     print()
#     print("index of highest maximum prob value:",K.get_value(K.argmax(K.max(box_scores, axis=1))))
#     print("highest maximum prob value:", K.get_value(K.max(box_scores)))
#     print("class index:", K.get_value(K.argmax(box_scores, axis=1))[K.get_value(K.argmax(K.max(box_scores, axis=1)))])
#     print("raw coordinates:", K.get_value(boxes)[K.get_value(K.argmax(K.max(box_scores, axis=1)))])
#     print("class index:", K.get_value(box_scores)[K.get_value(K.argmax(K.max(box_scores, axis=1)))])
    boxes_dummy = []
    scores_dummy = []
    classes_dummy = []
    scoremask_dummy = K.get_value(K.max(box_scores, axis=1) >= score_threshold)
    for i in range(scoremask_dummy.size):
        if scoremask_dummy[i]:
            boxes_dummy.append(K.get_value(boxes)[i].tolist())
            scores_dummy.append(K.get_value(K.max(box_scores, axis=1))[i].tolist())
            classes_dummy.append(K.get_value(K.argmax(box_scores, axis=1))[i].tolist())
    ''' # toggle to utilize either hacky solution with working classes output or IOU result-cutting solution with non-working classes output
    boxes_ = []
    scores_ = []
    classes_ = []
    mask = box_scores >= score_threshold
    max_boxes_tensor = K.constant(max_boxes, dtype='int32')
    for c in range(num_classes):
        # TODO: use keras backend instead of tf.
        class_boxes = tf.boolean_mask(boxes, mask[:, c])
        class_box_scores = tf.boolean_mask(box_scores[:, c], mask[:, c])
        nms_index = tf.image.non_max_suppression(
            class_boxes, class_box_scores, max_boxes_tensor, iou_threshold=iou_threshold)
        class_boxes = K.gather(class_boxes, nms_index)
        class_box_scores = K.gather(class_box_scores, nms_index)
        classes = K.ones_like(class_box_scores, 'int32') * c
        boxes_.append(class_boxes)
        scores_.append(class_box_scores)
        classes_.append(classes)
    boxes_ = K.concatenate(boxes_, axis=0)
    scores_ = K.concatenate(scores_, axis=0)
    classes_ = K.concatenate(classes_, axis=0)
    '''
    boxes_ = boxes_dummy
    scores_ = scores_dummy
    classes_ = classes_dummy
    '''
    # '''

    return boxes_, scores_, classes_

In [5]:
loadedModel = None
try:
    # '''
    with open("./saved_models/model_5_inferenceModel.json", "r") as jsonFile:
        jsonRead = jsonFile.read()
        # NOTE: this *will* take time, may cause OOM warnings
        loadedModel = model_from_json(jsonRead, custom_objects=customObjects)
        copyLoadModel = clone_model(loadedModel)
        loadedModel = copyLoadModel
        del copyLoadModel
    # '''
    print("Model loading successful")
    ''' # toggle comment to utilize by-name model (h5 and json are based on same model)
    loadedModel.load_weights("./saved_models/model_0_trainModel.h5", by_name=True, skip_mismatch=True)
    ''' # or not (e.g. h5 from model_0, json from other)
    loadedModel.load_weights("./saved_models/model_0_trainModel.h5", by_name=False)    
    # '''   
    print("Model weight loading attempt successful")
except Exception as e:
    print("Failed to load model/weight data:", e)

Model loading successful
Model weight loading attempt successful


In [6]:
if loadedModel is not None:
#     image_directory = "./test2017/000000000202.jpg"
#     image_directory = "./test2017/000000001371.jpg"
#     image_directory = "./test2017/000000000647.jpg"
#     image_directory = "./train2017/000000376608.jpg"
    image_directory = "./train2017/000000000036.jpg"
    image_data = []
    prepImage = prepare_image(image_directory, (448,448))
    print(prepImage.shape)
    image_data = np.expand_dims(prepImage,axis=0)
    processed_model_output = yolo_eval(
        yolo_outputs = loadedModel.predict(
            x = image_data,
            verbose=1,
            steps=1
        ),
        anchors=tinyYolo_anchors,
        num_classes=80,
        image_shape=(448,448),
        max_boxes=20,
        score_threshold=0.7,
        iou_threshold=0.5
    )
    print("Found", len(processed_model_output[0]), "objects")
    for i in range(len(processed_model_output[0])):
        print("Obj", i)
        print("Top, Left, Bottom, Right: ", processed_model_output[0][i])
        print("Prob:", processed_model_output[1][i])
        print("Class:", processed_model_output[2][i])
print("Completed inference")

(448, 448, 3)
1/1 [==============================] - 0s 425ms/step
Found 1350 objects
Obj 0
Top, Left, Bottom, Right:  [-25.0, -29.392675399780273, 57.000003814697266, 62.39234924316406]
Prob: 0.9977860450744629
Class: 56
Obj 1
Top, Left, Bottom, Right:  [-25.0, -11.426990509033203, 57.000003814697266, 106.42730712890625]
Prob: 0.9971468448638916
Class: 56
Obj 2
Top, Left, Bottom, Right:  [-25.0, 23.147441864013672, 57.000003814697266, 133.86131286621094]
Prob: 0.9945778846740723
Class: 56
Obj 3
Top, Left, Bottom, Right:  [-118.93196105957031, -1.9203910827636719, 159.70191955566406, 160.9207000732422]
Prob: 0.9939600229263306
Class: 16
Obj 4
Top, Left, Bottom, Right:  [-25.499839782714844, 61.486412048339844, 56.500160217285156, 172.20030212402344]
Prob: 0.9879599213600159
Class: 28
Obj 5
Top, Left, Bottom, Right:  [-110.03279876708984, 35.512481689453125, 151.71951293945312, 188.48751831054688]
Prob: 0.9934725165367126
Class: 16
Obj 6
Top, Left, Bottom, Right:  [-25.499839782714844, 

Top, Left, Bottom, Right:  [18.668285369873047, 230.37094116210938, 69.85310363769531, 265.12921142578125]
Prob: 0.9970122575759888
Class: 26
Obj 494
Top, Left, Bottom, Right:  [28.400577545166016, 253.85128784179688, 48.154197692871094, 274.14874267578125]
Prob: 0.9623072147369385
Class: 12
Obj 495
Top, Left, Bottom, Right:  [27.068002700805664, 276.04656982421875, 52.43215560913086, 296.343994140625]
Prob: 0.7876498699188232
Class: 7
Obj 496
Top, Left, Bottom, Right:  [14.157669067382812, 204.05718994140625, 65.34249114990234, 416.97747802734375]
Prob: 0.9988391399383545
Class: 67
Obj 497
Top, Left, Bottom, Right:  [-4.4497880935668945, 318.79388427734375, 78.716064453125, 336.706298828125]
Prob: 0.9893184900283813
Class: 34
Obj 498
Top, Left, Bottom, Right:  [-2.279818534851074, 341.5047607421875, 98.037353515625, 360.57244873046875]
Prob: 0.9990819096565247
Class: 41
Obj 499
Top, Left, Bottom, Right:  [12.257668495178223, 339.3074951171875, 66.74362182617188, 376.3074951171875]
Pro

Top, Left, Bottom, Right:  [268.40057373046875, 210.05552673339844, 288.1542053222656, 225.86318969726562]
Prob: 0.9940900802612305
Class: 29
Obj 994
Top, Left, Bottom, Right:  [256.6669616699219, 205.26986694335938, 301.83740234375, 235.94393920898438]
Prob: 0.99839848279953
Class: 78
Obj 995
Top, Left, Bottom, Right:  [268.6405029296875, 222.7938690185547, 288.3941345214844, 240.70628356933594]
Prob: 0.9979428648948669
Class: 36
Obj 996
Top, Left, Bottom, Right:  [269.0405578613281, 238.103759765625, 294.40472412109375, 254.930908203125]
Prob: 0.9574148058891296
Class: 77
Obj 997
Top, Left, Bottom, Right:  [-190.21495056152344, 218.20541381835938, 748.2252807617188, 273.8758544921875]
Prob: 0.9960506558418274
Class: 40
Obj 998
Top, Left, Bottom, Right:  [260.0388488769531, 251.25460815429688, 311.22369384765625, 288.2546081542969]
Prob: 0.7767482399940491
Class: 32
Obj 999
Top, Left, Bottom, Right:  [212.40243530273438, 237.8512420654297, 347.5975646972656, 290.1487731933594]
Prob: 0